# Section 3: Bayesian Time Series Models

#### PyData London 2025 - Bayesian Time Series Analysis with PyMC

---

In this section, we'll explore the landscape of Bayesian time series models, from simple random walks to sophisticated state-space models and Gaussian processes. These models provide the foundation for understanding temporal patterns and making probabilistic forecasts.

In [6]:
# Import necessary libraries for Section 3
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import pymc as pm
import pytensor.tensor as pt
import arviz as az
import warnings

# Configure plotting and suppress warnings
az.style.use("arviz-darkgrid")
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 100
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
RNG = np.random.default_rng(RANDOM_SEED:=42)

print("🔧 Libraries loaded successfully!")


🔧 Libraries loaded successfully!


In [2]:
# Load and prepare data (consistent with previous sections)
births_data = pl.read_csv('../data/births.csv', null_values=['null', 'NA', '', 'NULL'])
births_data = births_data.filter(pl.col('day').is_not_null())

monthly_births = (births_data
    .group_by(['year', 'month'])
    .agg(pl.col('births').sum())
    .sort(['year', 'month'])
)

births_subset = (monthly_births
    .filter((pl.col('year') >= 1970) & (pl.col('year') <= 1990))
    .with_row_index('index')
)

# Standardize the data
original_data = births_subset['births'].to_numpy()
births_standardized = (original_data - original_data.mean()) / original_data.std()
n_obs = len(births_standardized)

# Create time variables for regression models
time_idx = np.arange(n_obs)
time_normalized = (time_idx - time_idx.mean()) / time_idx.std()

print(f"📊 Data prepared: {n_obs} observations")
print(f"   Date range: {births_subset['year'].min()}-{births_subset['year'].max()}")
print(f"   Standardized data: mean={births_standardized.mean():.3f}, std={births_standardized.std():.3f}")

📊 Data prepared: 228 observations
   Date range: 1970-1988
   Standardized data: mean=-0.000, std=1.000


## Part 1: Gaussian Random Walks in PyMC

Random walks form the foundation of many time series models. They provide a natural way to model **smooth trends**, **level shifts**, and **time-varying parameters** while maintaining computational tractability and clear interpretation.

### Mathematical Foundation

A **Gaussian Random Walk** is defined as:

$$x_t = x_{t-1} + \epsilon_t$$

where $\epsilon_t \sim \mathcal{N}(0, \sigma^2)$ represents the **innovation** or step size.

With **drift** (systematic trend):
$$x_t = x_{t-1} + \mu + \epsilon_t$$

### Why Random Walks Excel for Time Series

**Flexibility** allows random walks to adapt to various trend patterns without imposing rigid functional forms. **Smoothness** ensures that changes occur gradually rather than abruptly, which matches many real-world processes. **Uncertainty propagation** naturally increases over time, reflecting our decreasing confidence in long-term predictions. **Parameter interpretability** makes the models easy to understand and communicate to stakeholders.

### Applications and Use Cases

Random walks excel at modeling **economic indicators** that evolve smoothly over time, **demographic trends** that change gradually, **environmental measurements** with natural variation, and **latent states** in more complex hierarchical models.

### Simple Random Walk

Let's start with the simplest time series model: a random walk without drift. A **Gaussian Random Walk** is defined mathematically as:

$$x_t = x_{t-1} + \epsilon_t$$

where $\epsilon_t \sim \mathcal{N}(0, \sigma_{\text{walk}}^2)$ represents the **innovation** or step size at each time period.

The key properties of this process include:

- **Non-stationarity**: The variance increases linearly with time as $\text{Var}(x_t) = t \cdot \sigma_{\text{walk}}^2$
- **Martingale property**: $\mathbb{E}[x_t | x_{t-1}, x_{t-2}, \ldots] = x_{t-1}$
- **Infinite memory**: All past shocks have permanent effects on the current level
- **No mean reversion**: The process has no tendency to return to any particular value

In our implementation, we separate the **latent random walk process** from the **observation process**, creating a state-space structure:

**State equation**: $x_t = x_{t-1} + \epsilon_t$ where $\epsilon_t \sim \mathcal{N}(0, \sigma_{\text{walk}}^2)$

**Observation equation**: $y_t = x_t + \nu_t$ where $\nu_t \sim \mathcal{N}(0, \sigma_{\text{obs}}^2)$

This separation allows us to distinguish between **process noise** (how much the underlying state changes) and **measurement noise** (how much error exists in our observations).

In [3]:
# Simple Random Walk
with pm.Model() as random_walk_model:
    # Step size of the random walk (innovation standard deviation)
    sigma_walk = pm.HalfNormal('sigma_walk', sigma=1.0)
    
    # Initial value distribution
    init_dist = pm.Normal.dist(mu=0, sigma=1)
    
    # Gaussian random walk process
    walk = pm.GaussianRandomWalk('walk', 
                                mu=0,  # no drift
                                sigma=sigma_walk, 
                                init_dist=init_dist,
                                steps=n_obs-1)
    
    # Observation noise (measurement error)
    sigma_obs = pm.HalfNormal('sigma_obs', sigma=1.0)
    
    # Likelihood: observed data are noisy observations of the walk
    y_pred = pm.Normal('y_pred', mu=walk, sigma=sigma_obs, observed=births_standardized)

# Sample from the model
with random_walk_model:
    trace_rw = pm.sample(1000, tune=1000, random_seed=RANDOM_SEED, chains=2)

print("🎯 **Random Walk Model Results**:")
print(az.summary(trace_rw, var_names=['sigma_walk', 'sigma_obs']))

# Extract and display key insights
sigma_walk_mean = az.extract(trace_rw)['sigma_walk'].mean().item()
sigma_obs_mean = az.extract(trace_rw)['sigma_obs'].mean().item()

print(f"\n📈 **Model Interpretation**:")
print(f"   • **Innovation variance**: {sigma_walk_mean:.3f} - controls trend smoothness")
print(f"   • **Observation noise**: {sigma_obs_mean:.3f} - measurement error level")
print(f"   • **Signal-to-noise ratio**: {sigma_walk_mean/sigma_obs_mean:.3f}")

Initializing NUTS using jitter+adapt_diag...
ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: local_subtensor_merge
ERROR (pytensor.graph.rewriting.basic): node: Subtensor{i}(Subtensor{start:stop}.0, 0)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/var/home/fonnesbeck/repos/ts_pydata_london_2025/.pixi/envs/default/lib/python3.13/site-packages/pytensor/graph/rewriting/basic.py", line 1922, in process_node
    replacements = node_rewriter.transform(fgraph, node)
  File "/var/home/fonnesbeck/repos/ts_pydata_london_2025/.pixi/envs/default/lib/python3.13/site-packages/pytensor/graph/rewriting/basic.py", line 1086, in transform
    return self.fn(fgraph, node)
           ~~~~~~~^^^^^^^^^^^^^^
  File "/var/home/fonnesbeck/repos/ts_pydata_london_2025/.pixi/envs/default/lib/python3.13/site-packages/pytensor/tensor/rewriting/subtensor.py", line 542, in local_subtensor_merge
    merge_two_slice

Output()

Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 2 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


🎯 **Random Walk Model Results**:
             mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
sigma_walk  0.382  0.040   0.308    0.455      0.002    0.001     501.0   
sigma_obs   0.329  0.035   0.258    0.389      0.002    0.001     514.0   

            ess_tail  r_hat  
sigma_walk     825.0    1.0  
sigma_obs      674.0    1.0  

📈 **Model Interpretation**:
   • **Innovation variance**: 0.382 - controls trend smoothness
   • **Observation noise**: 0.329 - measurement error level
   • **Signal-to-noise ratio**: 1.161


### Random Walk with Drift

Now let's add a **drift term** to capture any underlying systematic trend in the data. A **Random Walk with Drift** extends the basic random walk by including a deterministic trend component:

$$x_t = x_{t-1} + \mu + \epsilon_t$$

where $\mu$ is the **drift parameter** representing the expected change per time period, and $\epsilon_t \sim \mathcal{N}(0, \sigma_{\text{walk}}^2)$.

The mathematical properties of this process include:

- **Expected value**: $\mathbb{E}[x_t] = x_0 + \mu \cdot t$ (linear trend)
- **Variance**: $\text{Var}(x_t) = t \cdot \sigma_{\text{walk}}^2$ (increases with time)
- **Covariance**: $\text{Cov}(x_s, x_t) = \min(s,t) \cdot \sigma_{\text{walk}}^2$ for $s \leq t$

The drift parameter $\mu$ allows the model to capture **systematic directional movement** while maintaining the **stochastic flexibility** of random walks. Positive drift indicates an upward trend, negative drift indicates a downward trend, and zero drift reduces to the simple random walk.

This model is particularly useful for:
- **Economic time series** with underlying growth trends
- **Population dynamics** with systematic changes
- **Financial assets** with risk premiums
- **Climate variables** with long-term trends

In [5]:
# Random Walk with Drift
with pm.Model() as rw_drift_model:
    # Drift parameter (systematic trend)
    mu_drift = pm.Normal('mu_drift', mu=0, sigma=0.1)
    
    # Innovation variance (random fluctuations)
    sigma_walk = pm.HalfNormal('sigma_walk', sigma=0.5)
    
    # Random walk with drift
    rw_drift = pm.GaussianRandomWalk('rw_drift', 
                                    mu=mu_drift,  # Drift term
                                    sigma=sigma_walk,
                                    steps=n_obs-1)
    
    # Observation equation
    sigma_obs = pm.HalfNormal('sigma_obs', sigma=0.5)
    obs = pm.Normal('obs', mu=rw_drift, sigma=sigma_obs, observed=births_standardized)

# Sample from the model
with rw_drift_model:
    trace_rw_drift = pm.sample(1000, tune=1000, random_seed=RANDOM_SEED, chains=2)

print("🎯 **Random Walk with Drift Results**:")
print(az.summary(trace_rw_drift, var_names=['mu_drift', 'sigma_walk', 'sigma_obs']))

# Extract and interpret results
mu_drift_mean = az.extract(trace_rw_drift)['mu_drift'].mean().item()
mu_drift_hdi = az.hdi(trace_rw_drift, var_names=['mu_drift'])['mu_drift']

print(f"\n📈 **Drift Analysis**:")
print(f"   • **Estimated drift**: {mu_drift_mean:.4f} per time period")
print(f"   • **95% HDI**: [{mu_drift_hdi[0]:.4f}, {mu_drift_hdi[1]:.4f}]")
if mu_drift_hdi[0] > 0:
    print(f"   • **Interpretation**: Strong evidence for positive trend")
elif mu_drift_hdi[1] < 0:
    print(f"   • **Interpretation**: Strong evidence for negative trend")
else:
    print(f"   • **Interpretation**: No clear systematic trend detected")

Initializing NUTS using jitter+adapt_diag...
ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: local_subtensor_merge
ERROR (pytensor.graph.rewriting.basic): node: Subtensor{i}(Subtensor{start:stop}.0, 0)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/var/home/fonnesbeck/repos/ts_pydata_london_2025/.pixi/envs/default/lib/python3.13/site-packages/pytensor/graph/rewriting/basic.py", line 1922, in process_node
    replacements = node_rewriter.transform(fgraph, node)
  File "/var/home/fonnesbeck/repos/ts_pydata_london_2025/.pixi/envs/default/lib/python3.13/site-packages/pytensor/graph/rewriting/basic.py", line 1086, in transform
    return self.fn(fgraph, node)
           ~~~~~~~^^^^^^^^^^^^^^
  File "/var/home/fonnesbeck/repos/ts_pydata_london_2025/.pixi/envs/default/lib/python3.13/site-packages/pytensor/tensor/rewriting/subtensor.py", line 542, in local_subtensor_merge
    merge_two_slice

Output()

Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 3 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


🎯 **Random Walk with Drift Results**:
             mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
mu_drift    0.004  0.025  -0.041    0.052      0.000    0.001    4106.0   
sigma_walk  0.386  0.041   0.317    0.464      0.002    0.001     452.0   
sigma_obs   0.326  0.036   0.257    0.391      0.002    0.001     406.0   

            ess_tail  r_hat  
mu_drift      1592.0   1.00  
sigma_walk     830.0   1.01  
sigma_obs      622.0   1.01  

📈 **Drift Analysis**:
   • **Estimated drift**: 0.0041 per time period
   • **95% HDI**: [-0.0411, 0.0523]
   • **Interpretation**: No clear systematic trend detected


### Autoregressive Model AR(1)

**Autoregressive models** capture temporal dependence by relating current observations to past observations. An **AR(1) model** is defined as:

$$y_t = \phi y_{t-1} + \epsilon_t$$

where $\phi$ is the **autoregressive coefficient** and $\epsilon_t \sim \mathcal{N}(0, \sigma^2)$ is white noise.

**Key Mathematical Properties**:

- **Stationarity condition**: $|\phi| < 1$ ensures the process is stationary
- **Mean**: $\mathbb{E}[y_t] = 0$ (assuming zero mean)
- **Variance**: $\text{Var}(y_t) = \frac{\sigma^2}{1 - \phi^2}$ (constant for stationary process)
- **Autocorrelation**: $\rho(k) = \phi^k$ (exponential decay)

**Interpretation of $\phi$**:
- $\phi > 0$: **Positive persistence** - high values tend to be followed by high values
- $\phi < 0$: **Oscillatory behavior** - high values tend to be followed by low values
- $\phi \approx 1$: **Near unit root** - very persistent, close to random walk behavior
- $\phi = 0$: **White noise** - no temporal dependence

The **half-life** of shocks is approximately $\ln(0.5)/\ln(|\phi|)$ periods, indicating how long it takes for the effect of a shock to decay to half its initial impact.

AR models are fundamental building blocks that can be extended to:
- **AR(p)**: Higher-order autoregressive models
- **ARMA(p,q)**: Combined autoregressive and moving average models
- **VAR**: Vector autoregressive models for multiple time series

In [ ]:
# AR(1) Model
with pm.Model() as ar1_model:
    # AR coefficient (ensure stationarity)
    rho = pm.Beta('rho', alpha=1, beta=1)  # Beta(1,1) = Uniform(0,1)
    # Transform to (-1, 1) for AR coefficient
    phi = pm.Deterministic('phi', 2 * rho - 1)
    
    # Innovation variance
    sigma = pm.HalfNormal('sigma', sigma=1)
    
    # AR(1) process
    ar1 = pm.AR('ar1', rho=phi, sigma=sigma, constant=False, steps=n_obs-1)
    
    # Likelihood - observed data
    obs = pm.Normal('obs', mu=ar1, sigma=0.1, observed=births_standardized[1:])

# Sample from AR(1) model
with ar1_model:
    trace_ar1 = pm.sample(1000, tune=1000, random_seed=RANDOM_SEED, chains=2)

print("🎯 **AR(1) Model Results**:")
print(az.summary(trace_ar1, var_names=['phi', 'sigma']))

## Part 2: Bayesian Regression for Time Series

Bayesian regression models provide an explicit approach to decomposing time series into interpretable components. Unlike random walks that learn patterns implicitly, regression models allow us to specify **linear trends**, **polynomial patterns**, and **seasonal components** directly.

### Advantages of Explicit Component Modeling

**Interpretability** stands as the primary advantage—each parameter has a clear meaning that can be communicated to stakeholders. **Flexibility** allows us to easily add or remove components based on domain knowledge. **Prior incorporation** becomes natural when we understand what each parameter represents. **Forecasting** benefits from explicit trend and seasonal components that can be extrapolated with confidence.

### Mathematical Framework

The general form of a Bayesian regression time series model is:

$$y_t = \mu + \beta_1 f_1(t) + \beta_2 f_2(t) + \ldots + \beta_k f_k(t) + \epsilon_t$$

where $f_i(t)$ are **basis functions** that capture different temporal patterns, and $\epsilon_t \sim \mathcal{N}(0, \sigma^2)$.

### Linear Trend Model

The simplest regression approach models the time series as a **linear function of time** plus noise. The mathematical specification is:

$$y_t = \alpha + \beta t + \epsilon_t$$

where:
- $\alpha$ is the **intercept** (baseline level at time zero)
- $\beta$ is the **slope** (rate of change per time unit)
- $t$ is the **time index** (often centered for numerical stability)
- $\epsilon_t \sim \mathcal{N}(0, \sigma^2)$ is the **error term**

This model assumes:
- **Constant rate of change**: The trend is linear over time
- **Homoscedastic errors**: Constant variance across time
- **Independent errors**: No temporal correlation in residuals

The linear trend model works well when the underlying process has a consistent directional trend and is particularly useful for:
- **Economic indicators** with steady growth
- **Population growth** in stable environments
- **Technology adoption** in early phases
- **Climate variables** with monotonic changes

In [ ]:
# Linear Trend Model
with pm.Model() as linear_trend_model:
    # Intercept (baseline level)
    alpha = pm.Normal('alpha', mu=0, sigma=1)
    
    # Trend coefficient (change per time unit)
    beta_trend = pm.Normal('beta_trend', mu=0, sigma=1)
    
    # Expected value (linear trend)
    mu_t = pm.Deterministic('mu_t', alpha + beta_trend * time_normalized)
    
    # Observation noise
    sigma_obs = pm.HalfNormal('sigma_obs', sigma=1)
    
    # Likelihood
    y_obs = pm.Normal('y_obs', mu=mu_t, sigma=sigma_obs, observed=births_standardized)

# Sample from linear trend model
with linear_trend_model:
    trace_linear = pm.sample(1000, tune=1000, random_seed=RANDOM_SEED, chains=2)

### Polynomial Trend Model

When linear trends are insufficient, **polynomial terms** can capture more complex trend patterns. A **quadratic trend model** extends the linear model:

$$y_t = \alpha + \beta_1 t + \beta_2 t^2 + \epsilon_t$$

where:
- $\alpha$ is the **intercept**
- $\beta_1$ is the **linear coefficient** (initial rate of change)
- $\beta_2$ is the **quadratic coefficient** (acceleration/deceleration)
- $\epsilon_t \sim \mathcal{N}(0, \sigma^2)$ is the error term

**Interpretation of coefficients**:
- $\beta_2 > 0$: **Accelerating growth** (upward curvature)
- $\beta_2 < 0$: **Decelerating growth** (downward curvature)
- $\beta_2 = 0$: **Linear trend** (reduces to linear model)

The **instantaneous rate of change** at time $t$ is:
$$\frac{dy_t}{dt} = \beta_1 + 2\beta_2 t$$

**Cautions with polynomial models**:
- **Overfitting risk**: Higher-order polynomials can fit noise
- **Extrapolation problems**: Polynomials can behave poorly outside the data range
- **Multicollinearity**: Powers of time are often highly correlated
- **Interpretability**: Higher-order terms become difficult to interpret

**Best practices**: Use centered time variables, apply regularizing priors for higher-order terms, and validate extrapolation performance carefully.

In [ ]:
# Polynomial Trend Model (quadratic)
with pm.Model() as poly_trend_model:
    # Intercept
    alpha = pm.Normal('alpha', mu=0, sigma=1)
    
    # Linear trend coefficient
    beta_1 = pm.Normal('beta_1', mu=0, sigma=1)
    
    # Quadratic trend coefficient (use smaller prior variance)
    beta_2 = pm.Normal('beta_2', mu=0, sigma=0.5)
    
    # Expected value (quadratic trend)
    mu_t = pm.Deterministic('mu_t', 
                           alpha + 
                           beta_1 * time_normalized + 
                           beta_2 * time_normalized**2)
    
    # Observation noise
    sigma_obs = pm.HalfNormal('sigma_obs', sigma=1)
    
    # Likelihood
    y_obs = pm.Normal('y_obs', mu=mu_t, sigma=sigma_obs, observed=births_standardized)

# Sample from polynomial trend model
with poly_trend_model:
    trace_poly = pm.sample(1000, tune=1000, random_seed=RANDOM_SEED, chains=2)

### Seasonal Components with Fourier Terms

**Fourier terms** provide an elegant way to model seasonal patterns using trigonometric functions. For a time series with period $P$, we can represent seasonal patterns as:

$$s_t = \sum_{k=1}^{K} \left[ \beta_{s,k} \sin\left(\frac{2\pi k t}{P}\right) + \beta_{c,k} \cos\left(\frac{2\pi k t}{P}\right) \right]$$

where:
- $P$ is the **seasonal period** (e.g., 12 for monthly data with annual seasonality)
- $K$ is the **number of harmonics** (typically $K \leq P/2$)
- $\beta_{s,k}, \beta_{c,k}$ are the **Fourier coefficients**

**Mathematical properties**:
- **Periodicity**: $s_{t+P} = s_t$ for all $t$
- **Orthogonality**: Different harmonics are orthogonal over complete periods
- **Amplitude**: $A_k = \sqrt{\beta_{s,k}^2 + \beta_{c,k}^2}$ for harmonic $k$
- **Phase**: $\phi_k = \arctan(\beta_{s,k}/\beta_{c,k})$ for harmonic $k$

**Advantages of Fourier representation**:
- **Flexibility**: Can approximate any periodic function
- **Parsimony**: Few parameters can capture complex seasonal patterns
- **Interpretability**: Each harmonic represents a specific frequency component
- **Computational efficiency**: Fast evaluation and differentiation

**Harmonic interpretation**:
- **First harmonic** ($k=1$): Fundamental seasonal cycle
- **Second harmonic** ($k=2$): Half-period cycles (e.g., bi-annual patterns)
- **Higher harmonics**: Capture finer seasonal details and asymmetries

The complete model combines trend and seasonality:
$$y_t = \alpha + \beta t + s_t + \epsilon_t$$

In [ ]:
# Trend + Seasonality with Fourier Terms
# Create seasonal components (annual cycle for monthly data)
period = 12  # Monthly data with annual seasonality
seasonal_freq = 2 * np.pi / period
sin_seasonal = np.sin(seasonal_freq * time_idx)
cos_seasonal = np.cos(seasonal_freq * time_idx)

# Add second harmonic for more flexible seasonality
sin_seasonal_2 = np.sin(2 * seasonal_freq * time_idx)
cos_seasonal_2 = np.cos(2 * seasonal_freq * time_idx)

with pm.Model() as seasonal_model:
    # Intercept
    mu_overall = pm.Normal('mu_overall', mu=0, sigma=1)
    
    # Trend coefficient
    beta_trend = pm.Normal('beta_trend', mu=0, sigma=1)
    
    # First harmonic seasonal coefficients
    beta_sin_1 = pm.Normal('beta_sin_1', mu=0, sigma=1)
    beta_cos_1 = pm.Normal('beta_cos_1', mu=0, sigma=1)
    
    # Second harmonic seasonal coefficients
    beta_sin_2 = pm.Normal('beta_sin_2', mu=0, sigma=0.5)
    beta_cos_2 = pm.Normal('beta_cos_2', mu=0, sigma=0.5)
    
    # Expected value (trend + seasonality)
    mu_t = pm.Deterministic('mu_t', 
                           mu_overall + 
                           beta_trend * time_normalized + 
                           beta_sin_1 * sin_seasonal + 
                           beta_cos_1 * cos_seasonal +
                           beta_sin_2 * sin_seasonal_2 + 
                           beta_cos_2 * cos_seasonal_2)
    
    # Observation noise
    sigma_obs = pm.HalfNormal('sigma_obs', sigma=1)
    
    # Likelihood
    y_obs = pm.Normal('y_obs', mu=mu_t, sigma=sigma_obs, observed=births_standardized)

# Sample from seasonal model
with seasonal_model:
    trace_seasonal = pm.sample(1000, tune=1000, random_seed=RANDOM_SEED, chains=2)

print("🎯 **Seasonal Model Results**:")
print(az.summary(trace_seasonal, var_names=['mu_overall', 'beta_trend', 'beta_sin_1', 'beta_cos_1', 'sigma_obs']))

# Calculate seasonal amplitude
beta_sin_1_mean = az.extract(trace_seasonal)['beta_sin_1'].mean().item()
beta_cos_1_mean = az.extract(trace_seasonal)['beta_cos_1'].mean().item()
seasonal_amplitude = np.sqrt(beta_sin_1_mean**2 + beta_cos_1_mean**2)

print(f"\n📈 **Seasonal Analysis**:")
print(f"   • **Seasonal amplitude**: {seasonal_amplitude:.3f}")
print(f"   • **Seasonal strength**: {seasonal_amplitude / births_standardized.std():.1%} of total variation")
if seasonal_amplitude > 0.2:
    print(f"   • **Interpretation**: Strong seasonal pattern detected")
else:
    print(f"   • **Interpretation**: Weak seasonal pattern")

## Part 3: State-Space Models in PyMC

State-space models represent one of the most powerful frameworks for time series analysis. They separate the **latent state** (unobserved process) from the **observation process** (what we actually measure), enabling sophisticated modeling of complex temporal dynamics.

### Mathematical Framework

State-space models consist of two equations:

**State equation** (evolution): $x_t = f(x_{t-1}, \theta) + \eta_t$

**Observation equation** (measurement): $y_t = g(x_t, \phi) + \epsilon_t$

where $x_t$ represents the **latent state**, $y_t$ is the **observation**, $\eta_t$ and $\epsilon_t$ are **noise terms**, and $\theta, \phi$ are **parameters**.

### Advantages of State-Space Modeling

**Latent variable modeling** allows us to separate the true underlying process from measurement noise and observation errors. **Missing data handling** becomes natural since we can estimate latent states even when observations are missing. **Hierarchical structure** enables modeling of complex relationships between multiple time series. **Uncertainty propagation** flows naturally through the state evolution, providing realistic prediction intervals.

### Local Level Model (Dynamic Linear Model)

The **local level model** is the simplest state-space model, consisting of two equations:

**State equation**: $\mu_t = \mu_{t-1} + \eta_t$ where $\eta_t \sim \mathcal{N}(0, \sigma_{\eta}^2)$

**Observation equation**: $y_t = \mu_t + \epsilon_t$ where $\epsilon_t \sim \mathcal{N}(0, \sigma_{\epsilon}^2)$

where:
- $\mu_t$ is the **latent level** (unobserved state)
- $\eta_t$ is the **state innovation** (level changes)
- $\epsilon_t$ is the **observation noise** (measurement error)
- $\sigma_{\eta}^2$ controls **state variability**
- $\sigma_{\epsilon}^2$ controls **observation noise**

**Key properties**:
- **Signal-to-noise ratio**: $q = \sigma_{\eta}^2 / \sigma_{\epsilon}^2$ determines smoothness
- **High $q$**: Flexible tracking of level changes (noisy estimates)
- **Low $q$**: Smooth level estimates (slow adaptation)
- **Kalman filtering**: Optimal recursive estimation of latent states

**Applications**:
- **Trend extraction** from noisy economic indicators
- **Signal processing** in engineering applications
- **Baseline estimation** for anomaly detection
- **Missing data interpolation** using state estimates

This model excels at **trend estimation** and **signal extraction** from noisy data while naturally handling missing observations.

In [ ]:
# Local Level Model (State-Space)
with pm.Model() as local_level_model:
    # Process noise (state evolution variance)
    sigma_level = pm.HalfNormal('sigma_level', sigma=0.5)
    
    # Observation noise (measurement error variance)
    sigma_obs = pm.HalfNormal('sigma_obs', sigma=0.5)
    
    # Initial level
    init_level = pm.Normal('init_level', mu=0, sigma=1)
    
    # Level process (latent state - random walk)
    init_dist = pm.Normal.dist(mu=init_level, sigma=sigma_level)
    level = pm.GaussianRandomWalk('level',
                                 mu=0,  # no drift in level
                                 sigma=sigma_level,
                                 init_dist=init_dist,
                                 steps=n_obs-1)
    
    # Observations (noisy measurements of latent state)
    obs = pm.Normal('obs', mu=level, sigma=sigma_obs, observed=births_standardized)

# Sample from local level model
with local_level_model:
    trace_local_level = pm.sample(1000, tune=1000, random_seed=RANDOM_SEED, chains=2)

print("🎯 **Local Level Model Results**:")
print(az.summary(trace_local_level, var_names=['sigma_level', 'sigma_obs', 'init_level']))

# Analyze signal-to-noise ratio
sigma_level_mean = az.extract(trace_local_level)['sigma_level'].mean().item()
sigma_obs_mean = az.extract(trace_local_level)['sigma_obs'].mean().item()
signal_noise_ratio = sigma_level_mean / sigma_obs_mean

print(f"\n📈 **State-Space Analysis**:")
print(f"   • **Process noise**: {sigma_level_mean:.3f} - latent state variability")
print(f"   • **Observation noise**: {sigma_obs_mean:.3f} - measurement error")
print(f"   • **Signal-to-noise ratio**: {signal_noise_ratio:.3f}")
if signal_noise_ratio > 1:
    print(f"   • **Interpretation**: High signal relative to noise - clear underlying trend")
else:
    print(f"   • **Interpretation**: High noise relative to signal - noisy measurements")

### Local Linear Trend Model

The **local linear trend model** extends the local level model by adding a **time-varying slope** component. This model consists of three equations:

**Level equation**: $\mu_t = \mu_{t-1} + \beta_{t-1} + \eta_{\mu,t}$ where $\eta_{\mu,t} \sim \mathcal{N}(0, \sigma_{\mu}^2)$

**Trend equation**: $\beta_t = \beta_{t-1} + \eta_{\beta,t}$ where $\eta_{\beta,t} \sim \mathcal{N}(0, \sigma_{\beta}^2)$

**Observation equation**: $y_t = \mu_t + \epsilon_t$ where $\epsilon_t \sim \mathcal{N}(0, \sigma_{\epsilon}^2)$

where:
- $\mu_t$ is the **time-varying level**
- $\beta_t$ is the **time-varying slope** (trend)
- $\eta_{\mu,t}$ is the **level innovation**
- $\eta_{\beta,t}$ is the **slope innovation**
- $\epsilon_t$ is the **observation noise**

**Key features**:
- **Adaptive trending**: The slope can change over time
- **Smooth evolution**: Both level and slope follow random walks
- **Flexible growth**: Can model accelerating, decelerating, or changing direction
- **Parameter interpretation**: $\sigma_{\beta}^2$ controls how much the growth rate varies

**Special cases**:
- $\sigma_{\beta}^2 = 0$: **Constant slope** (reduces to linear trend)
- $\sigma_{\mu}^2 = 0$: **Deterministic level** given trend
- $\sigma_{\beta}^2 \gg \sigma_{\mu}^2$: **Highly variable growth rates**

This model allows both the level and the trend to evolve over time, providing greater flexibility for modeling changing growth rates in economic, demographic, or technological time series.

In [ ]:
# Local Linear Trend Model
with pm.Model() as local_trend_model:
    # Process noise variances
    sigma_level = pm.HalfNormal('sigma_level', sigma=0.3)  # level innovation
    sigma_trend = pm.HalfNormal('sigma_trend', sigma=0.1)  # trend innovation
    
    # Observation noise
    sigma_obs = pm.HalfNormal('sigma_obs', sigma=0.5)
    
    # Initial conditions
    init_level = pm.Normal('init_level', mu=0, sigma=1)
    init_trend = pm.Normal('init_trend', mu=0, sigma=0.1)
    
    # Trend process (random walk)
    trend_init_dist = pm.Normal.dist(mu=init_trend, sigma=sigma_trend)
    trend = pm.GaussianRandomWalk('trend',
                                 mu=0,
                                 sigma=sigma_trend,
                                 init_dist=trend_init_dist,
                                 steps=n_obs-1)
    
    # Level process (random walk + trend)
    # This is more complex - we'll use a simpler approximation
    level_innovation = pm.Normal('level_innovation', mu=0, sigma=sigma_level, shape=n_obs)
    
    # Construct level with trend
    level = pm.Deterministic('level', 
                            pt.cumsum(pt.concatenate([[init_level], 
                                                     trend + level_innovation[1:]])))
    
    # Observations
    obs = pm.Normal('obs', mu=level, sigma=sigma_obs, observed=births_standardized)

# Sample from local trend model
with local_trend_model:
    trace_local_trend = pm.sample(1000, tune=1000, random_seed=RANDOM_SEED, chains=2)

print("🎯 **Local Linear Trend Model Results**:")
print(az.summary(trace_local_trend, var_names=['sigma_level', 'sigma_trend', 'sigma_obs']))

# Analyze trend variability
sigma_trend_mean = az.extract(trace_local_trend)['sigma_trend'].mean().item()
print(f"\n📈 **Trend Dynamics Analysis**:")
print(f"   • **Trend variability**: {sigma_trend_mean:.4f}")
if sigma_trend_mean > 0.05:
    print(f"   • **Interpretation**: Highly variable trend - changing growth rates")
else:
    print(f"   • **Interpretation**: Stable trend - consistent growth pattern")

## Part 4: Stochastic Volatility Models

**Stochastic volatility models** address the reality that uncertainty itself changes over time. These models are essential for financial time series where periods of calm alternate with periods of high volatility, but they're also valuable for any time series where the variability changes systematically.

### Mathematical Framework

The standard stochastic volatility model consists of:

**Returns equation**: $r_t = \mu + \sigma_t \epsilon_t$

**Volatility equation**: $\log(\sigma_t^2) = \log(\sigma_{t-1}^2) + \nu_t$

where $\epsilon_t, \nu_t \sim \mathcal{N}(0,1)$ are **independent innovations**.

### Applications Beyond Finance

While originally developed for financial markets, stochastic volatility models apply to many domains. **Environmental data** often shows changing variability due to seasonal effects or climate change. **Economic indicators** may have periods of stability and instability. **Biological processes** can exhibit changing variance due to external stressors or developmental stages.

In [ ]:
# Generate synthetic data with time-varying volatility for demonstration
np.random.seed(42)
n_periods = 200
true_mu = 0.02
true_tau = 0.1

# Simulate log-volatility as random walk
log_vol = np.cumsum(np.random.normal(0, true_tau, n_periods))
vol = np.exp(log_vol / 2)

# Simulate returns with time-varying volatility
returns = np.random.normal(true_mu, vol)

print(f"📈 Generated {n_periods} synthetic returns with stochastic volatility")
print(f"   Mean return: {returns.mean():.4f}")
print(f"   Return volatility: {returns.std():.4f}")
print(f"   Volatility range: [{vol.min():.3f}, {vol.max():.3f}]")

### Stochastic Volatility Model

**Stochastic volatility models** capture **time-varying uncertainty** by allowing the variance to follow its own stochastic process. The standard specification is:

**Returns equation**: $r_t = \mu + \sigma_t \epsilon_t$ where $\epsilon_t \sim \mathcal{N}(0, 1)$

**Log-volatility equation**: $\log(\sigma_t) = \log(\sigma_{t-1}) + \nu_t$ where $\nu_t \sim \mathcal{N}(0, \tau^2)$

where:
- $r_t$ are the **returns** (or observations)
- $\mu$ is the **mean return**
- $\sigma_t$ is the **time-varying volatility**
- $\tau^2$ controls the **volatility of volatility**
- $\epsilon_t, \nu_t$ are **independent** innovations

**Mathematical properties**:
- **Log-normal volatility**: $\sigma_t$ is always positive
- **Volatility clustering**: High volatility periods tend to persist
- **Heavy tails**: Returns distribution has fatter tails than normal
- **Leverage effect**: Can be extended to include correlation between return and volatility shocks

**Parameter interpretation**:
- **Large $\tau$**: Highly variable volatility (regime switching)
- **Small $\tau$**: Stable volatility (close to constant variance)
- **Persistence**: $\mathbb{E}[\log(\sigma_{t+1}) | \log(\sigma_t)] = \log(\sigma_t)$ (unit root in log-volatility)

**Applications beyond finance**:
- **Environmental data**: Changing measurement precision
- **Economic indicators**: Periods of stability vs. uncertainty
- **Biological processes**: Variable response to treatments
- **Engineering systems**: Time-varying noise characteristics

In [ ]:
# Stochastic Volatility Model
with pm.Model() as stoch_vol_model:
    # Mean return parameter
    mu = pm.Normal('mu', mu=0, sigma=0.1)
    
    # Volatility process parameters
    tau = pm.HalfNormal('tau', sigma=0.2)  # Innovation in log-volatility
    
    # Log-volatility random walk
    init_dist = pm.Normal.dist(mu=np.log(0.1), sigma=1)
    log_sigma = pm.GaussianRandomWalk('log_sigma',
                                     mu=0,
                                     sigma=tau,
                                     init_dist=init_dist,
                                     steps=n_periods-1)
    
    # Convert to volatility (ensure positivity)
    sigma = pm.Deterministic('sigma', pm.math.exp(log_sigma))
    
    # Likelihood (returns with time-varying volatility)
    y_pred = pm.Normal('y_pred', mu=mu, sigma=sigma, observed=returns)

# Sample from stochastic volatility model
with stoch_vol_model:
    trace_sv = pm.sample(1000, tune=2000, target_accept=0.9, random_seed=RANDOM_SEED, chains=2)

print("🎯 **Stochastic Volatility Model Results**:")
print(az.summary(trace_sv, var_names=['mu', 'tau']))

# Analyze volatility dynamics
tau_mean = az.extract(trace_sv)['tau'].mean().item()
sigma_posterior = az.extract(trace_sv)['sigma']
vol_persistence = np.corrcoef(sigma_posterior[:-1], sigma_posterior[1:])[0,1]

print(f"\n📈 **Volatility Analysis**:")
print(f"   • **Volatility innovation**: {tau_mean:.4f}")
print(f"   • **Volatility persistence**: {vol_persistence:.3f}")
print(f"   • **Mean volatility**: {sigma_posterior.mean():.3f}")
if tau_mean > 0.1:
    print(f"   • **Interpretation**: Highly variable volatility - regime changes")
else:
    print(f"   • **Interpretation**: Stable volatility - gradual changes")

## Part 5: Gaussian Process Regression for Time Series

**Gaussian Processes (GPs)** provide the ultimate flexibility in time series modeling by learning the temporal structure directly from data without imposing strong parametric assumptions. They excel when the functional form is unknown or when we want to capture complex, non-linear patterns.

### Key Concepts

**Covariance functions** (kernels) define the relationship between points in time, encoding our assumptions about smoothness, periodicity, and other structural properties. **Length scales** control how quickly correlations decay with time separation. **Marginal variance** controls the overall variability of the process. **Hyperparameter learning** allows the model to adapt these properties to the data.

### Advantages of GP Modeling

**Non-parametric flexibility** means we don't need to specify a particular functional form in advance. **Uncertainty quantification** is built-in, providing credible intervals that naturally widen in regions with less data. **Kernel composition** allows combining different temporal patterns (trends, seasonality, noise) in principled ways. **Extrapolation** provides reasonable predictions beyond the training data with appropriate uncertainty.

In [ ]:
# Prepare data for GP regression (use subset for computational efficiency)
n_gp = 100
X_gp = np.arange(n_gp)[:, None]  # Time points (must be 2D for GP)
y_gp = births_standardized[:n_gp]  # Observations

print(f"📊 GP data prepared: {n_gp} observations")
print(f"   Time range: 0 to {n_gp-1}")
print(f"   Data range: [{y_gp.min():.3f}, {y_gp.max():.3f}]")

### Gaussian Process with RBF Kernel

We'll start with a **Radial Basis Function (RBF)** kernel, also known as the squared exponential kernel. The RBF kernel is defined as:

$$k(t_i, t_j) = \sigma_f^2 \exp\left(-\frac{(t_i - t_j)^2}{2\ell^2}\right)$$

where:
- $\sigma_f^2$ is the **signal variance** (marginal variance)
- $\ell$ is the **length scale** (correlation distance)
- $t_i, t_j$ are **time points**

**Mathematical properties**:
- **Smoothness**: Infinitely differentiable (very smooth functions)
- **Stationarity**: Covariance depends only on $|t_i - t_j|$
- **Isotropy**: Same behavior in all directions
- **Decay**: Correlation decreases exponentially with distance

**Hyperparameter interpretation**:
- **Large $\ell$**: Long-range correlations, smooth trends
- **Small $\ell$**: Short-range correlations, rapid changes
- **Large $\sigma_f^2$**: High signal variance, large deviations from mean
- **Small $\sigma_f^2$**: Low signal variance, stays close to mean

The complete GP model includes observation noise:
$$y_t = f(t) + \epsilon_t$$
where $f(t) \sim \mathcal{GP}(0, k(t_i, t_j))$ and $\epsilon_t \sim \mathcal{N}(0, \sigma_n^2)$.

This kernel assumes smooth, continuous functions and is excellent for capturing non-linear trends without sharp discontinuities.

In [ ]:
# Gaussian Process with RBF Kernel
with pm.Model() as gp_model:
    # GP hyperparameters
    length_scale = pm.HalfNormal('length_scale', sigma=10.0)  # temporal correlation
    eta = pm.HalfNormal('eta', sigma=1.0)  # marginal standard deviation
    
    # Define the covariance function (RBF/Squared Exponential)
    cov_func = eta**2 * pm.gp.cov.ExpQuad(1, ls=length_scale)
    
    # GP prior
    gp = pm.gp.Marginal(cov_func=cov_func)
    
    # Observation noise
    sigma_gp = pm.HalfNormal('sigma_gp', sigma=0.5)
    
    # Observed data
    y_pred = gp.marginal_likelihood('y_pred', X=X_gp, y=y_gp, sigma=sigma_gp)

# Sample from the model
with gp_model:
    trace_gp = pm.sample(1000, tune=1000, random_seed=RANDOM_SEED, chains=2)

print("🎯 **Gaussian Process Model Results**:")
print(az.summary(trace_gp, var_names=['length_scale', 'eta', 'sigma_gp']))

# Interpret GP hyperparameters
length_scale_mean = az.extract(trace_gp)['length_scale'].mean().item()
eta_mean = az.extract(trace_gp)['eta'].mean().item()
sigma_gp_mean = az.extract(trace_gp)['sigma_gp'].mean().item()

print(f"\n📈 **GP Analysis**:")
print(f"   • **Length scale**: {length_scale_mean:.2f} time units")
print(f"   • **Signal variance**: {eta_mean:.3f}")
print(f"   • **Noise variance**: {sigma_gp_mean:.3f}")
print(f"   • **Signal-to-noise ratio**: {eta_mean/sigma_gp_mean:.2f}")
if length_scale_mean > 20:
    print(f"   • **Interpretation**: Long-range correlations - smooth trends")
elif length_scale_mean > 5:
    print(f"   • **Interpretation**: Medium-range correlations - moderate smoothness")
else:
    print(f"   • **Interpretation**: Short-range correlations - rapid changes")

## Model Comparison and Selection

With multiple models fitted to our data, we need principled methods for comparing their performance and selecting the most appropriate approach. **Bayesian model comparison** provides several tools for this purpose, each with different strengths and applications.

In [ ]:
# Compare models using WAIC (Widely Applicable Information Criterion)
# Note: We'll compare models that use the same data
models_dict = {
    'Random Walk': trace_rw,
    'RW with Drift': trace_rw_drift,
    'AR(1)': trace_ar1,
    'Linear Trend': trace_linear,
    'Polynomial Trend': trace_poly,
    'Seasonal Model': trace_seasonal,
    'Local Level': trace_local_level
}

# Compute WAIC for model comparison
comparison = az.compare(models_dict, ic='waic')
print("🎯 **Model Comparison Results (WAIC)**:")
print(comparison)

# Visualize model comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot WAIC comparison
az.plot_compare(comparison, ax=ax1)
ax1.set_title('Model Comparison: WAIC Scores')

# Plot model weights
model_names = comparison.index
weights = comparison['weight']
ax2.bar(range(len(model_names)), weights, color='steelblue', alpha=0.7)
ax2.set_xticks(range(len(model_names)))
ax2.set_xticklabels(model_names, rotation=45, ha='right')
ax2.set_ylabel('Model Weight')
ax2.set_title('Model Weights (Probability)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Interpret results
best_model = comparison.index[0]
best_weight = comparison.loc[best_model, 'weight']

print(f"\n📊 **Model Selection Insights**:")
print(f"   • **Best model**: {best_model} (weight: {best_weight:.3f})")
print(f"   • **WAIC interpretation**: Lower values indicate better predictive performance")
print(f"   • **Model weights**: Represent relative probability of each model")
if best_weight > 0.5:
    print(f"   • **Conclusion**: Strong evidence favors the {best_model} model")
else:
    print(f"   • **Conclusion**: Model uncertainty - consider ensemble approaches")

## Summary and Key Insights

In this comprehensive section, we've explored the rich landscape of Bayesian time series models, from fundamental building blocks to sophisticated state-space and non-parametric approaches. Each model type offers unique advantages and is suited to different types of temporal patterns and modeling objectives.

### Model Categories and Applications

**Gaussian Random Walks** serve as the foundation for many time series models, providing smooth trend estimation and natural uncertainty propagation. These models excel when we need to capture gradual changes over time without imposing rigid functional forms. The ability to add drift terms makes them suitable for trending data, while their computational efficiency makes them practical for large datasets.

**Bayesian Regression Models** offer maximum interpretability by explicitly modeling trend and seasonal components. The linear trend model provides a baseline for understanding directional movement, while polynomial extensions capture more complex curvature patterns. Fourier-based seasonal models elegantly handle periodic patterns and can accommodate multiple seasonal frequencies simultaneously.

**State-Space Models** represent the most flexible framework for separating latent dynamics from observation processes. The local level model excels at signal extraction from noisy data, while local linear trend models capture time-varying growth rates. These models naturally handle missing data and provide a foundation for more complex hierarchical structures.

**Stochastic Volatility Models** address the crucial reality that uncertainty itself changes over time. While originally developed for financial applications, these models apply broadly to any domain where variability patterns are of interest. They provide insights into regime changes and help identify periods of stability versus instability.

**Gaussian Process Models** offer ultimate flexibility by learning temporal structure directly from data. Their non-parametric nature makes them ideal for exploratory analysis and situations where the functional form is unknown. The ability to compose kernels allows for sophisticated modeling of multiple temporal patterns simultaneously.

### Practical Modeling Guidelines

**Start simple** and build complexity gradually. Random walks and linear trends provide excellent baselines that often perform surprisingly well. **Consider the data characteristics** when choosing models—trending data benefits from drift terms or explicit trend modeling, while seasonal data requires appropriate periodic components.

**Use domain knowledge** to guide model selection and prior specification. Understanding the underlying process generating your data helps choose appropriate model structures and realistic parameter ranges. **Validate thoroughly** using posterior predictive checks and out-of-sample evaluation to ensure models capture essential patterns without overfitting.

**Embrace uncertainty** in model selection. When multiple models perform similarly, consider ensemble approaches that average across models weighted by their posterior probabilities. This approach often provides more robust predictions than relying on a single "best" model.

### Computational Considerations

**Model complexity** directly impacts computational requirements. Simple models like random walks and linear regression scale well to large datasets, while Gaussian processes become computationally intensive for more than a few hundred observations. **Prior specification** significantly affects convergence—weakly informative priors often provide the best balance between regularization and flexibility.

**Convergence diagnostics** remain crucial regardless of model complexity. Always examine R-hat values, effective sample sizes, and trace plots to ensure reliable inference. **Computational efficiency** can often be improved through data standardization, appropriate parameterizations, and careful prior choices.

**Next**: In the following sections, we'll explore model evaluation techniques, forecasting methods, and practical applications that build upon these foundational modeling approaches.

---

**Key Takeaways**:
- Bayesian time series modeling offers a rich toolkit for capturing temporal patterns while quantifying uncertainty
- Random walks provide flexible trend modeling, regression approaches offer interpretable component decomposition
- State-space models separate signal from noise, stochastic volatility captures changing uncertainty
- Gaussian processes provide non-parametric flexibility for complex temporal patterns
- Model selection should balance complexity with interpretability, always validated through rigorous posterior checking and out-of-sample evaluation
- The Bayesian framework naturally handles uncertainty in both parameters and model structure, providing robust foundations for forecasting and decision-making under uncertainty